# Sasquatch TAP example
This notebook creates initial topics and schemas to start on the event-driven TAP development.

### Get the Kafka cluster ID
To create Kafka topics we are going to use the REST Proxy API v3. To use that API you need is to get the Kafka cluster ID. We are going to use the IDF development environment.

In [ ]:
import requests 

sasquatch_rest_proxy_url = "https://data-dev.lsst.cloud/sasquatch-rest-proxy"

headers = {"content-type": "application/json"}

r = requests.get(f"{sasquatch_rest_proxy_url}/v3/clusters", headers=headers)

cluster_id = r.json()['data'][0]['cluster_id']

print(cluster_id)

### Job run
Schema and payload for the job-run topic

In [ ]:
import json

value_schema = json.dumps({"namespace": "lsst.tap", 
                           "type": "record", 
                           "name": "jobRun",  
                           "fields": [{"name": "jobID", "type": "string", "doc": "UWS job identifier"}, 
                                      {"name": "query", "type": "string", "doc": "The SQL query to be executed by QServ"}
                                      ]
                          })

payload = {
    "value_schema": value_schema,
    "records": [
        {
            "value": {
                "jobID": "1",
                "query": "SELECT TOP 10 * FROM table",
            }
        },
        {
            "value": {
                "jobID": "2",
                "query": "SELECT TOP 10 * FROM table",
            }
        }
    ],
}
payload

This is the URL for sending data to our topic


In [ ]:
url = f"{sasquatch_rest_proxy_url}/topics/lsst.tap.job-run"
url

Make a POST request to the topic endpoint to send data:

In [ ]:
headers = {
    "Content-Type": "application/vnd.kafka.avro.v2+json",
    "Accept": "application/vnd.kafka.v2+json",
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

### Job Status
Schema and payload for the job-status topic


In [ ]:
import json

value_schema = json.dumps({"namespace": "lsst.tap", 
                           "type": "record", 
                           "name": "jobStatus",  
                            "fields": [
                            {
                              "name": "jobID",
                              "type": "string",
                              "doc": "UWS job identifier"
                            },
                            {
                              "name": "qservID",
                              "type": "string",
                              "doc": "QServ query ID"
                            },
                            {
                              "name": "status",
                              "type": {
                                "type": "enum",
                                "name": "jobStatusEnum",
                                "symbols": ["QUEUED", "EXECUTING", "COMPLETED", "ERROR", "ABORTED", "DELETED"]
                              },
                               "doc": "Current status of the job",
                            }
                          ]
})

payload = {
    "value_schema": value_schema,
    "records": [
        {
            "value": {
                "jobID": "1",
                "qservID": "1",
                "status": "EXECUTING",
            }
        },
        {
            "value": {
                "jobID": "2",
                "qservID": "2",
                "status": "QUEUED",
            }
        }
    ],
}
payload

In [ ]:
url = f"{sasquatch_rest_proxy_url}/topics/lsst.tap.job-status"
url

In [ ]:
headers = {
    "Content-Type": "application/vnd.kafka.avro.v2+json",
    "Accept": "application/vnd.kafka.v2+json",
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)